In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

In [11]:
img_cv = cv2.imread('./Samples/107.jpg')

cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
type(img_cv)

numpy.ndarray

In [15]:
img_cv

array([[[ 23,  23,  23],
        [ 23,  23,  23],
        [ 23,  23,  23],
        ...,
        [ 21,  24,  23],
        [ 21,  24,  23],
        [ 21,  24,  23]],

       [[160, 160, 160],
        [160, 160, 160],
        [160, 160, 160],
        ...,
        [161, 163, 163],
        [161, 163, 163],
        [161, 163, 163]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [253, 253, 251],
        [253, 253, 251],
        [253, 253, 251]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [254, 254, 254],
        [254, 254, 254],
        [254, 254, 254]]

### Text from Image

In [16]:
text_cv = pytesseract.image_to_string(img_cv)

In [8]:
print(text_cv)

 

Test Result

Normal Ranges

 

[COMPLETE BLOOD COUNT]
[HAEMOGLOBIN HAEMATOCRIT]

 

 

HAEMOGLOBIN 11.5 g/dl
HAEMATOCRIT 38.0 %
RBC. 6.15 x10E12/L
MC. 61.8 fl
M.C.H. 18.7 pg
M.C.H.C. 30.3 g/dL
RD.W 19.8 %
WB.c. 12.5 x10E9/L
NEUTROPHILS 62.5 %
LYMPHOCYTES, 27.0 %
EOSINOPHILS 19 %
MONOCYTES 8.0 %
BASOPHILS 0.6 %
Neutrophils lymphocytes ratio (NLR) 2.3 ratio

 

PLATELETS 476 x10E9/L

METHODOLOGY:
The test is performed on Automated Haematology Analyzer.

PERIPHERAL FILM oe .
ANISOCYTOSIS, HYPOCHROMIC, MICROCYTIC
POIKILOCYTOSIS, POLYCHROMASIA
ELLIPTICAL CELLS, TARGET CELLS
FEW FRAGMENTED RBC
? IRON DEFICIENCY, ? THALASSEMIA MINOR
SUGGEST SERUM IRON/TIBC OR FERRITIN LEVEL

 

 

 

 

 

 

 

 

(11-14.5)
(34.5-45.4)

(3.61-5.2)
(78.1-95.3)
(25.3-31.7)
(30.3-34.4)
(12.1-16.9)

(4.6-10.8)
(34.9-76.2)
(17.5-45)
(0.3-7.4)
(3.9-10)
(0-1)

(1-4)
(154-433)

NOTE: SUGGEST HEMOGLOBIN VARIANT ANALYSIS BY HPLC OR HB ELECTROPHORESIS TO RULE OUT
THALASSEMIA MINOR IF IRON STUDIES ARE WITHIN NORMAL L

### Image to Data

In [17]:
data = pytesseract.image_to_data(img_cv)

In [18]:
#data.split('\n')
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:],columns=dataList[0])

In [19]:
df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,640,661,-1,
1,2,1,1,0,0,0,45,15,568,375,-1,
2,3,1,1,1,0,0,55,15,545,160,-1,
3,4,1,1,1,1,0,55,15,538,28,-1,
4,5,1,1,1,1,1,55,15,30,28,69.247932,‘WBC
...,...,...,...,...,...,...,...,...,...,...,...,...
205,2,1,6,0,0,0,30,656,602,5,-1,
206,3,1,6,1,0,0,30,656,602,5,-1,
207,4,1,6,1,1,0,30,656,602,5,-1,
208,5,1,6,1,1,1,30,656,602,5,95.000000,


In [20]:
df.dropna(inplace=True) # drop the missing in rows
col_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height']
df[col_int] = df[col_int].astype(int)
df['conf'] = df['conf'].astype(float)

In [21]:
df.dtypes

level          int64
page_num       int64
block_num      int64
par_num        int64
line_num       int64
word_num       int64
left           int64
top            int64
width          int64
height         int64
conf         float64
text          object
dtype: object

In [ ]:
image = img_cv.copy()
level = 'word'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    #print(l,x,y,w,h,c)
    if level == 'page':
        if l == 1:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
            
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        else:
            continue
    
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        else:
            continue
    
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        else:
            continue
            
    elif level == 'word':
        if l == 5:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
            
cv2.imshow("bounding box",image)
cv2.waitKey()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/devious/Documents/Projects/blood-test-tracker/env/lib64/python3.10/site-packages/cv2/qt/plugins"
